# Srock Strategy - Factor Strategy 0012 - Industry Rotation

## 1. Import Library

In [1]:
import dai
import random
import pandas as pd

## 2. Prepare Strategy Data

### (1) Industry Data

In [2]:
sd = '2024-09-30'
ed = '2026-02-13'

In [3]:
sql_industry = f"""
SELECT
    date,
    industry_level1_code AS sw2021_level1,
    c_pct_rank(close / m_lag(close,  5)) AS _ret_05,
    c_pct_rank(close / m_lag(close, 20)) AS _ret_20,
    c_pct_rank(close / m_lag(close, 60)) AS _ret_60,
    c_rank(-1 * (_ret_05 + _ret_20 + _ret_60)) AS rank_ind
FROM cn_stock_industry_sw_bar1d JOIN cn_stock_industry_index_mapper on cn_stock_industry_sw_bar1d.instrument = cn_stock_industry_index_mapper.index_level1_code
QUALIFY rank_ind <= 5
ORDER BY date, rank_ind
"""

In [4]:
df_industry = dai.query(sql_industry, filters = {"date":[sd, ed]}).df()
df_industry

,date,sw2021_level1,rank_ind
0,2024-12-30,270000,1.0
1,2024-12-30,480000,2.0
2,2024-12-30,730000,3.0
3,2024-12-30,450000,4.5
4,2024-12-30,510000,4.5
...,...,...,...
1345,2026-02-13,510000,1.0
1346,2026-02-13,610000,2.0
1347,2026-02-13,720000,3.0
1348,2026-02-13,240000,4.5


### (2) Stock Data

In [5]:
sql_stock = f"""
SELECT 
    date, 
    instrument,
    sw2021_level1,
    c_pct_rank(float_market_cap) * 0.5 + c_pct_rank(-1*pe_ttm) * 0.25 + c_pct_rank(-1*ps_ttm) * 0.25 AS factor,
    RANK_EXT(factor) OVER (PARTITION BY date, sw2021_level1) AS _rank_stock,
FROM cn_stock_prefactors
WHERE is_risk_warning = 0
AND pe_ttm > 0
AND ps_ttm > 0
QUALIFY _rank_stock <= 3
ORDER BY date
"""

In [6]:
df_stock = dai.query(sql_stock, filters = {"date":[sd, ed]}).df()
df_stock

,date,instrument,sw2021_level1,factor
0,2024-09-30,920726.BJ,340000,0.165502
1,2024-09-30,603696.SH,340000,0.245870
2,2024-09-30,001318.SZ,340000,0.216709
3,2024-09-30,300234.SZ,610000,0.261436
4,2024-09-30,301429.SZ,610000,0.261626
...,...,...,...,...
31138,2026-02-13,603402.SH,460000,0.135793
31139,2026-02-13,920122.BJ,460000,0.101190
31140,2026-02-13,603323.SH,480000,0.723509
31141,2026-02-13,601860.SH,480000,0.719584


### (3) Combined Strategy Data

In [7]:
sql_combined = f"""
WITH
data_industry AS (
    {sql_industry}
),
data_stock AS (
    {sql_stock}
)
SELECT
    date,
    instrument,
    factor
FROM data_industry JOIN data_stock USING (date, sw2021_level1)
"""

In [8]:
sql_trade = f"""
WITH
data_strategy AS (
    {sql_combined}
),
data_filter AS (
    SELECT
        date,
        instrument,
        factor AS score,
        c_rank(factor) AS score_rank,
    FROM data_strategy
    QUALIFY score_rank <= 15
),
data_date AS (
    SELECT
        date,
        instrument,
        score, 
        score_rank, 
        1 / c_sum(1) AS position, 
    FROM data_filter JOIN mldt_cn_stock_calendar_daily USING (date)
    WHERE is_week_end_trade = 1
)
SELECT *
FROM data_date
ORDER BY date, score_rank
"""

In [9]:
df_strategy = dai.query(sql_trade, filters = {"date":[sd, ed]}).df()
df_strategy

,date,instrument,score,score_rank,position
0,2024-12-31,301177.SZ,0.088300,1.0,0.066667
1,2024-12-31,301314.SZ,0.094287,2.0,0.066667
2,2024-12-31,301419.SZ,0.098581,3.0,0.066667
3,2024-12-31,920374.BJ,0.101314,4.0,0.066667
4,2024-12-31,920454.BJ,0.103722,5.0,0.066667
...,...,...,...,...,...
888,2026-02-13,300467.SZ,0.228676,11.0,0.066667
889,2026-02-13,600678.SH,0.237964,12.0,0.066667
890,2026-02-13,600689.SH,0.269819,13.0,0.066667
891,2026-02-13,600620.SH,0.300824,14.0,0.066667


## 3. Backtest Strategy

In [10]:
from bigmodule import M

def BigTrader_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))

def BigTrader_Before_Trading(context, data):
    pass

def BigTrader_Handle_Tick(context, tick):
    pass

def BigTrader_Handle_Data(context, data):

    df_now = context.data[context.data["date"] == data.current_dt.strftime("%Y-%m-%d")]

    if len(df_now) == 0:
        return
    
    target_instruments  = list(df_now["instrument"])
    holding_instruments = list(context.get_account_positions().keys())

    for instrument in holding_instruments:
        if instrument not in target_instruments:
            context.order_target_percent(instrument, 0)
        
    for i, x in df_now.iterrows():
        position = 0.0 if pd.isnull(x.position) else float(x.position)
        context.order_target_percent(x.instrument, position)

def BigTrader_Handle_Trade(context, trade):
    pass

def BigTrader_Handle_Order(context, order):
    pass

def BigTrader_After_Trading(context, data):
    pass

BigTrader = M.bigtrader.v34(
    
    data = df_strategy,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_Initialize,
    before_trading_start = BigTrader_Before_Trading,
    handle_tick          = BigTrader_Handle_Tick,
    handle_data          = BigTrader_Handle_Data,
    handle_trade         = BigTrader_Handle_Trade,
    handle_order         = BigTrader_Handle_Order,
    after_trading        = BigTrader_After_Trading,
    
    capital_base = 1000000  + random.uniform(0, 10),
    frequency="""daily""",
    product_type="""自动""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""close""",
    order_price_field_sell="""close""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader"""
) 

[2026-02-19 21:46:58] [info     ] bigtrader.v34 开始运行 ..
[2026-02-19 21:46:58] [info     ] 2024-12-31, 2026-02-13, , , instruments=185
[2026-02-19 21:46:58] [info     ] bigtrader module V2.2.0
[2026-02-19 21:46:58] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10
order_target_percent: price is null for '000633.SZ' at '2025-06-27 15:00:00'
[2026-02-19 21:47:00] [info     ] backtest done, raw_perf_ds:dai.DataSource("_03092d1de1ae4f19ad4d82351b1e7878")


[2026-02-19 21:47:01] [info     ] bigtrader.v34 运行完成 [3.210s].
